In [1]:
from qdrant_client import QdrantClient, models

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
qd_client = QdrantClient("http://localhost:6333")

In [3]:
EMBEDDING_DIMENSIONALITY = 512


In [4]:
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [6]:
from fastembed import TextEmbedding

In [12]:
import requests

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

In [14]:
# Define the collection name
collection_name = "zoomcamp-homework"

# Create the collection with specified vector parameters
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [15]:
points = []
id = 0

for course in documents_raw:
    for doc in course['documents']:

        point = models.PointStruct(
            id=id,
            vector=models.Document(text=doc['text'], model=model_handle), #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
            payload={
                "text": doc['text'],
                "section": doc['section'],
                "course": course['course']
            } #save all needed metadata fields
        )
        points.append(point)

        id += 1

In [17]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [27]:
def search(query, limit=1):

    results = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle 
        ),
        limit=limit, # top closest matches
        with_vectors=True,
        with_payload=True #to get metadata in the results
        
    )

    return results

In [29]:
response = search('I just discovered the course. Can I join now?')

In [40]:
min(response.points[0].vector)

-0.10889833

In [48]:
import numpy as np
q = response.points[0].vector
np.linalg.norm(q)

np.float64(0.9999999988807053)

In [49]:
np.dot(q, q)

np.float64(0.9999999977614107)

In [57]:
doc = 'Can I still join the course after the start date?'

point = models.PointStruct(
            id=id,
            vector=models.Document(text=doc, model=model_handle), #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
        )

In [61]:
# Define the collection name
collection_name = "zoomcamp-doc-q2"

# Create the collection with specified vector parameters
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [63]:
qd_client.upsert(
    collection_name=collection_name,
    points=[point]
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [64]:
response = search('I just discovered the course. Can I join now?')


In [65]:
response

QueryResponse(points=[ScoredPoint(id=948, version=0, score=0.9008529, payload={}, vector=[-0.05453039, -0.07834517, 0.031361036, 0.023423493, -0.030632148, -0.055104163, 0.022472648, 0.015038616, -0.018115958, 0.018059535, -0.0048089228, -0.07573959, -0.0048727216, -0.09077178, 0.048394088, -0.04263174, -0.026146252, -0.044812914, -0.020692602, -0.037905447, -0.033032935, -0.008028502, 0.011856035, -0.03837212, 0.07263781, -0.058281176, -0.07308902, 0.034091137, 0.056070536, 0.050268475, -0.059929334, 0.05330166, -0.017952451, 0.004177451, -0.02454279, 0.013228825, 0.046417832, 0.010996376, -0.010149964, -0.0057752416, -0.00028398776, 0.049149636, 0.11121484, -0.027630642, -0.067892395, -0.018592827, -0.12396321, 0.012574593, 0.016529871, 0.00722243, -0.027723745, 0.051807355, -0.05986048, 0.05273394, 0.010384407, 0.04820913, -0.022902327, 0.015879998, -0.048051156, 0.010248942, 0.013788453, 0.038744498, -0.057989437, 0.05691391, -0.031520545, -0.026749857, -0.044805452, -0.040970754, 

In [66]:
q2 = response.points[0].vector
np.dot(q, q2)

np.float64(0.8638822239594446)

In [67]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [ ]:
# Define the collection name
collection_name = "zoomcamp-q3"

# Create the collection with specified vector parameters
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

In [86]:
points = []
id = 0

for doc in documents:
    point = models.PointStruct(
        id=id,
        vector=models.Document(text=doc['text'], model=model_handle), #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
        payload={
            "text": doc['text'],
            "section": doc['section'],
            "course": course['course']
        } #save all needed metadata fields
    )
    points.append(point)

    id += 1

In [87]:
points[0]

PointStruct(id=0, vector=Document(text="Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", model='jinaai/jina-embeddings-v2-small-en', options=None), payload={'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'course': 'mlops-zoomcamp'})

In [72]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [73]:
def search(query, limit=1):

    results = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle 
        ),
        limit=limit, # top closest matches
        with_vectors=True,
        with_payload=True #to get metadata in the results
        
    )

    return results

In [78]:
query = 'I just discovered the course. Can I join now?'

In [79]:
search(query)

QueryResponse(points=[ScoredPoint(id=1, version=0, score=0.8182378, payload={'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.', 'section': 'General course-related questions', 'course': 'mlops-zoomcamp'}, vector=[-0.059470892, -0.08523984, 0.0129208835, 0.028512737, -0.02816201, -0.040005114, 0.008416977, 0.051821243, -0.026958339, -0.011239185, 0.005320356, -0.063366205, 0.01483236, -0.09491826, 0.042256217, -0.045051593, -0.0017500113, -0.033401098, -0.06472191, -0.037634823, -0.059090544, -0.0023075656, 0.055512384, -0.018641295, 0.09813543, -0.05908135, -0.056841217, 0.07473078, 0.047964424, 0.028064102, -0.041140694, 0.065692306, -0.011913598, 0.008643482, -0.025080383, -0.012139176, 0.033481337, 0.021419918, -0.03224748, -0.06463362, -0.0

In [ ]:
# Define the collection name
collection_name = "zoomcamp-q4"

# Create the collection with specified vector parameters
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

In [88]:
points = []
id = 0

for doc in documents:
    point = models.PointStruct(
        id=id,
        vector=models.Document(text=doc['question'] + ' ' + doc['text'], model=model_handle), #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
        payload={
            "text": doc['text'],
            "section": doc['section'],
            "course": course['course']
        } #save all needed metadata fields
    )
    points.append(point)

    id += 1

In [89]:
points[0]

PointStruct(id=0, vector=Document(text="Course - Can I still join the course after the start date? Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", model='jinaai/jina-embeddings-v2-small-en', options=None), payload={'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'course': 'mlops-zoomcamp'})

In [91]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [92]:
search(query)

QueryResponse(points=[ScoredPoint(id=0, version=0, score=0.8514543, payload={'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'course': 'mlops-zoomcamp'}, vector=[-0.03958747, -0.06904071, 0.011466174, 0.02626039, -0.024734093, -0.024720704, 0.003439712, 0.042785935, -0.024162956, -0.026199229, -0.014921463, -0.078817844, -0.024740098, -0.07740146, 0.07686429, -0.04656838, -0.030845454, -0.044565715, -0.04481412, -0.060655635, -0.036711317, -0.01800093, 0.031952262, -0.06805735, 0.059340063, -0.07107091, -0.08380715, 0.07134353, 0.07235555, 0.024347804, -0.05510248, 0.029459156, -0.01830824, 0.025609976, -0.03172867, 0.027675109, 0.049635842, 0.013883932, -0.020100303, -0.0028251305, -0.029371912, 0.028263267, 0.100648426, 0.0028943736, -0.07165612, 0.0007678592, -0.080

In [98]:
from fastembed import TextEmbedding
models_list = TextEmbedding.list_supported_models()
import operator

print(min(models_list, key=operator.itemgetter('dim')))

{'model': 'BAAI/bge-small-en', 'sources': {'hf': 'Qdrant/bge-small-en', 'url': 'https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz', '_deprecated_tar_struct': True}, 'model_file': 'model_optimized.onnx', 'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.', 'license': 'mit', 'size_in_GB': 0.13, 'additional_files': [], 'dim': 384, 'tasks': {}}


In [99]:
model_handle = "BAAI/bge-small-en"

In [100]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [109]:
# Define the collection name
collection_name = "zoomcamp-q6"
EMBEDDING_DIMENSIONALITY=384

# Create the collection with specified vector parameters
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [112]:
points = []
id = 0

for doc in documents:
    point = models.PointStruct(
        id=id,
        vector=models.Document(text = doc['question'] + ' ' + doc['text'], model=model_handle), #embed text locally with "BAAI/bge-small-en" from FastEmbed
        payload={
            "text": doc['text'],
            "section": doc['section'],
            "course": course['course']
        } #save all needed metadata fields
    )
    points.append(point)

    id += 1

In [113]:
documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp'}

In [114]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [115]:
search(query)

QueryResponse(points=[ScoredPoint(id=14, version=1, score=0.87031734, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'course': 'mlops-zoomcamp'}, vector=[-0.022328932, -0.050472237, -0.027709432, -0.03740207, -0.012660511, 0.011062007, 0.0028257293, -0.015284091, -0.037188776, -0.017788403, 0.017320452, -0.045489114, 0.02576105, -0.004898703, 0.03174624, -0.0060602687, -0.034617264, -0.0011483284, -0.05011883, 0.010960391, -0.0071789343, -0.007407182, -0.011910453, 0.0029656515, -0.008808807, 0.039636776, -0.016958902, -0.06290091, -0.035196856, -0.16416648, -0.044827446, -0.040522013, -0.020155983,